# 라이브러리 임포트

In [ ]:
import numpy as np
import os
import pandas as pd
import tensorflow as tf
from tensorflow import keras

# 워킹 디렉터리 설정

In [ ]:
# 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/Toy/21~23년/230911 MAIC ECG AI Challenge 2023

/content/drive/MyDrive/Colab Notebooks/Toy/21~23년/230911 MAIC ECG AI Challenge 2023


In [ ]:
pwd

'/content/drive/MyDrive/Colab Notebooks/Toy/21~23년/230911 MAIC ECG AI Challenge 2023'

아래 폴더 공유 방법으로는 데이터 가져오기 어려움

In [ ]:
# ECG_adult_numpy_train (.npy 포함 폴더)
# https://drive.google.com/drive/folders/1MTPpMkgcQNi4fZqHpC468hZWhbusYAT-?usp=sharing

# ECG_adult_numpy_valid (.npy 포함 폴더)
# https://drive.google.com/drive/folders/1Mjgq3blHNF9VmvL11GScgIZz0hQt2kg2?usp=sharing

# ECG_child_numpy_train (.npy 포함 폴더)
# https://drive.google.com/drive/folders/1MVkELdZNLgY7UIRo_8p5iZQSJ2kV7R0k?usp=sharing

# ECG_child_numpy_valid (.npy 포함 폴더)
# https://drive.google.com/drive/folders/1M_DdseTnR4Zt2z8bF8uUsdFkRfX_Cxvh?usp=sharing

# ECG_adult_age_train.csv
# https://docs.google.com/spreadsheets/d/1UHIjtDBssp0OkTM3LHqScUDqV54tjoM5AOb2nZSIef0/edit?usp=sharing

# ECG_child_age_train.csv
# https://docs.google.com/spreadsheets/d/1NigRh0j7gRyTkV9NkGNw_VW6YueYE-p-TRapoz6RnnA/edit?usp=sharing

## 어른 경로 지정

In [ ]:
# 라벨 데이터 로딩
label_data = pd.read_csv('ECG_adult_age_train.csv')

train_folder = 'ECG_adult_numpy_train'
valid_folder = 'ECG_adult_numpy_valid'

## 어른 데이터 사이즈 확인

시간 부족으로 생략

In [ ]:
def identify_problematic_files(folder_path):
    for file_name in os.listdir(folder_path):
        try:
            data = np.load(os.path.join(folder_path, file_name))
            if data.shape != (60000,):
                print(f"File {file_name} has an unexpected shape {data.shape}.")
        except Exception as e:
            print(f"An exception occurred while loading {file_name}: {e}")

identify_problematic_files(train_folder)
identify_problematic_files(valid_folder)


KeyboardInterrupt: ignored

In [ ]:
# 데이터 형태 확인
sample_data = np.load(os.path.join(train_folder, 'ecg_adult_0.npy'))
print("Sample data shape:", sample_data.shape)

Sample data shape: (60000,)


In [ ]:
# 첫 5개 파일만 로드하여 형태 확인
for i in range(5):
    data = np.load(os.path.join(train_folder, f'ecg_adult_{i}.npy'))
    print(f"Shape of ecg_adult_{i}.npy: {data.shape}")

Shape of ecg_adult_0.npy: (60000,)
Shape of ecg_adult_1.npy: (60000,)
Shape of ecg_adult_2.npy: (60000,)
Shape of ecg_adult_3.npy: (60000,)
Shape of ecg_adult_4.npy: (60000,)


## 어른 데이터 로딩

In [ ]:
def load_data(folder_path, label_data):
    X = []
    y = []
    for file_name in os.listdir(folder_path):
        if file_name == "ecg_adult_16901.npy":  # 추가된 부분
            print(f"Skipping problematic file {file_name}")
            continue
        try:
            data = np.load(os.path.join(folder_path, file_name))
            if data.shape != (60000,):
                print(f"File {file_name} has an unexpected shape {data.shape}. Skipping.")
                continue
            X.append(data)
            y.append(label_data.loc[label_data['FILENAME'] == file_name.split('.')[0], 'AGE'].iloc[0])
        except Exception as e:
            print(f"An exception occurred while loading {file_name}: {e}. Skipping.")
            continue
    return np.array(X), np.array(y)

16901 데이터 다운 받다가 손상됐을 수 있음

In [ ]:
# Train 데이터 로딩
X_train, y_train = load_data(train_folder, label_data)

Skipping problematic file ecg_adult_16901.npy


## Train 셋, Test 셋 나누기

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Data Augmentation

메모리 터져서 이게 가장 현실적인 것 같음

In [ ]:
def data_generator(X, y, batch_size=32, augment_fn=None):
    num_samples = len(X)
    while True:  # Loop forever so the generator never terminates
        for offset in range(0, num_samples, batch_size):
            batch_X = X[offset:offset+batch_size]
            batch_y = y[offset:offset+batch_size]

            if augment_fn:
                batch_X, batch_y = augment_fn(batch_X, batch_y)

            yield batch_X, batch_y

# Augmentation 함수를 정의
def batch_augment(X_batch, y_batch):
    return apply_augmentation(X_batch, y_batch)  # 앞서 정의한 apply_augmentation 함수를 사용

# Generator 생성
train_gen = data_generator(X_train, y_train, batch_size=32, augment_fn=batch_augment)
valid_gen = data_generator(X_valid, y_valid, batch_size=32)

# 모델 학습

전이학습 모델 가져올 시간 부족

In [ ]:
from tensorflow.keras import callbacks

# 모델 구성
model = keras.Sequential([
    keras.layers.InputLayer(input_shape=(60000,)),
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(1)
])

# Early Stopping과 Model Checkpoint
early_stopping = callbacks.EarlyStopping(patience=5, restore_best_weights=True)
model_checkpoint = callbacks.ModelCheckpoint('best_model.h5', save_best_only=True)

# 모델 컴파일
model.compile(optimizer='adam',
              loss='mean_squared_error',
              metrics=['mae'])

# Keras 모델 훈련
model.fit(train_gen,
          steps_per_epoch=len(X_train) // 32,
          epochs=20,  # 시간 부족으로 에포크 수 50 까지 안됨
          validation_data=valid_gen,
          validation_steps=len(X_valid) // 32,
          callbacks=[early_stopping, model_checkpoint])  # 콜백 추가

Epoch 1/20
871/872 [============================>.] - ETA: 0s - loss: 15442.8105 - mae: 75.7143

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


872/872 [==============================] - 49s 49ms/step - loss: 15426.8242 - mae: 75.6652 - val_loss: 1883.4559 - val_mae: 39.1396
Epoch 2/20
872/872 [==============================] - 43s 49ms/step - loss: 2196.0596 - mae: 37.5839 - val_loss: 1268.4163 - val_mae: 31.0773
Epoch 3/20
872/872 [==============================] - 41s 47ms/step - loss: 1942.0496 - mae: 34.9887 - val_loss: 1385.5383 - val_mae: 33.0949
Epoch 4/20
872/872 [==============================] - 40s 46ms/step - loss: 1746.4207 - mae: 33.0302 - val_loss: 1408.6583 - val_mae: 33.4208
Epoch 5/20
872/872 [==============================] - 39s 45ms/step - loss: 1529.6421 - mae: 30.9473 - val_loss: 1449.4314 - val_mae: 34.1582
Epoch 6/20
872/872 [==============================] - 40s 46ms/step - loss: 1286.6549 - mae: 28.8365 - val_loss: 1610.8351 - val_mae: 36.3935
Epoch 7/20
872/872 [==============================] - 41s 47ms/step - loss: 1325.9437 - mae: 28.0631 - val_loss: 1292.5083 - val_mae: 32.0338


# 예측

In [ ]:
import pandas as pd
import numpy as np
import os

# 1. 테스트 데이터 로드
test_folder = '/content/drive/MyDrive/Colab Notebooks/Toy/21~23년/230911 MAIC ECG AI Challenge 2023/ECG_adult_numpy_valid'
X_test = []
test_files = []

for file_name in os.listdir(test_folder):
    data = np.load(os.path.join(test_folder, file_name))
    if data.shape == (60000,):
        X_test.append(data)
        test_files.append(file_name.split('.')[0])

X_test = np.array(X_test)

# 2. 예측 수행
y_pred = model.predict(X_test)

# 반올림 적용
y_pred_rounded = np.round(y_pred).astype(int)

# 3. submission.csv 파일 업데이트
submission_path = '/content/drive/MyDrive/Colab Notebooks/Toy/21~23년/230911 MAIC ECG AI Challenge 2023/submission.csv'
submission_df = pd.read_csv(submission_path)

for idx, file_name in enumerate(test_files):
    submission_df.loc[submission_df['FILENAME'] == file_name, 'AGE'] = y_pred_rounded[idx]

submission_df.to_csv(submission_path, index=False)


137/137 [==============================] - 1s 4ms/step


## 아이 경로 지정

In [ ]:
# 라벨 데이터 로딩
label_data = pd.read_csv('ECG_child_age_train.csv')

train_folder = 'ECG_child_numpy_train'
valid_folder = 'ECG_child_numpy_valid'

## 아이 데이터 사이즈 확인

In [ ]:
def identify_problematic_files(folder_path):
    for file_name in os.listdir(folder_path):
        try:
            data = np.load(os.path.join(folder_path, file_name))
            if data.shape != (60000,):
                print(f"File {file_name} has an unexpected shape {data.shape}.")
        except Exception as e:
            print(f"An exception occurred while loading {file_name}: {e}")

identify_problematic_files(train_folder)
identify_problematic_files(valid_folder)


In [ ]:
# 데이터 형태 확인
sample_data = np.load(os.path.join(train_folder, 'ecg_child_0.npy'))
print("Sample data shape:", sample_data.shape)

Sample data shape: (60000,)


In [ ]:
# 첫 5개 파일만 로드하여 형태 확인
for i in range(5):
    data = np.load(os.path.join(train_folder, f'ecg_child_{i}.npy'))
    print(f"Shape of ecg_child_{i}.npy: {data.shape}")

Shape of ecg_child_0.npy: (60000,)
Shape of ecg_child_1.npy: (60000,)
Shape of ecg_child_2.npy: (60000,)
Shape of ecg_child_3.npy: (60000,)
Shape of ecg_child_4.npy: (60000,)


## 아이 데이터 로딩

In [ ]:
def load_data(folder_path, label_data):
    X = []
    y = []
    for file_name in os.listdir(folder_path):
        if file_name == "ecg_adult_16901.npy":  # 추가된 부분
            print(f"Skipping problematic file {file_name}")
            continue
        try:
            data = np.load(os.path.join(folder_path, file_name))
            if data.shape != (60000,):
                print(f"File {file_name} has an unexpected shape {data.shape}. Skipping.")
                continue
            X.append(data)
            y.append(label_data.loc[label_data['FILENAME'] == file_name.split('.')[0], 'AGE'].iloc[0])
        except Exception as e:
            print(f"An exception occurred while loading {file_name}: {e}. Skipping.")
            continue
    return np.array(X), np.array(y)

In [ ]:
# Train 데이터 로딩
X_train, y_train = load_data(train_folder, label_data)

## Train 셋, Test 셋 나누기

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Data Augmentation

메모리 터져서 이게 가장 현실적인 것 같음

In [ ]:
def data_generator(X, y, batch_size=32, augment_fn=None):
    num_samples = len(X)
    while True:  # Loop forever so the generator never terminates
        for offset in range(0, num_samples, batch_size):
            batch_X = X[offset:offset+batch_size]
            batch_y = y[offset:offset+batch_size]

            if augment_fn:
                batch_X, batch_y = augment_fn(batch_X, batch_y)

            yield batch_X, batch_y

# Augmentation 함수를 정의
def batch_augment(X_batch, y_batch):
    return apply_augmentation(X_batch, y_batch)  # 앞서 정의한 apply_augmentation 함수를 사용

# Generator 생성
train_gen = data_generator(X_train, y_train, batch_size=32, augment_fn=batch_augment)
valid_gen = data_generator(X_valid, y_valid, batch_size=32)

# 모델 학습

전이학습 모델 가져올 시간 부족

In [ ]:
from tensorflow.keras import callbacks

# 모델 구성
model = keras.Sequential([
    keras.layers.InputLayer(input_shape=(60000,)),
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(1)
])

# Early Stopping과 Model Checkpoint
early_stopping = callbacks.EarlyStopping(patience=5, restore_best_weights=True)
model_checkpoint = callbacks.ModelCheckpoint('best_model.h5', save_best_only=True)

# 모델 컴파일
model.compile(optimizer='adam',
              loss='mean_squared_error',
              metrics=['mae'])

# Keras 모델 훈련
model.fit(train_gen,
          steps_per_epoch=len(X_train) // 32,
          epochs=20,  # 시간 부족으로 에포크 수 50 까지 안됨
          validation_data=valid_gen,
          validation_steps=len(X_valid) // 32,
          callbacks=[early_stopping, model_checkpoint])  # 콜백 추가

Epoch 1/20
218/219 [============================>.] - ETA: 0s - loss: 342823.4375 - mae: 334.1474

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


219/219 [==============================] - 14s 55ms/step - loss: 341637.4688 - mae: 333.5190 - val_loss: 35060.1250 - val_mae: 102.0132
Epoch 2/20
219/219 [==============================] - 10s 45ms/step - loss: 130806.0703 - mae: 196.5013 - val_loss: 5869.7988 - val_mae: 44.8950
Epoch 3/20
219/219 [==============================] - 10s 45ms/step - loss: 96088.5469 - mae: 138.3369 - val_loss: 3701.2163 - val_mae: 32.0663
Epoch 4/20
219/219 [==============================] - 10s 46ms/step - loss: 46627.6250 - mae: 94.7395 - val_loss: 1035.8156 - val_mae: 17.0999
Epoch 5/20
219/219 [==============================] - 9s 40ms/step - loss: 17822.5176 - mae: 56.8933 - val_loss: 1662.2549 - val_mae: 9.3466
Epoch 6/20
219/219 [==============================] - 10s 48ms/step - loss: 16773.4551 - mae: 44.9850 - val_loss: 120.4888 - val_mae: 7.0947
Epoch 7/20
219/219 [==============================] - 9s 43ms/step - loss: 8418.5381 - mae: 30.4522 - val_loss: 86.4007 - val_mae: 4.1589
Epoch 8/20
2

# 예측

In [ ]:
import pandas as pd
import numpy as np
import os

# 1. 테스트 데이터 로드
test_folder = '/content/drive/MyDrive/Colab Notebooks/Toy/21~23년/230911 MAIC ECG AI Challenge 2023/ECG_child_numpy_valid'
X_test = []
test_files = []

for file_name in os.listdir(test_folder):
    data = np.load(os.path.join(test_folder, file_name))
    if data.shape == (60000,):
        X_test.append(data)
        test_files.append(file_name.split('.')[0])

X_test = np.array(X_test)

# 2. 예측 수행
y_pred = model.predict(X_test)

# 반올림 적용 (소수점 첫째 자리까지)
y_pred_rounded = np.round(y_pred, decimals=1)

# 3. submission.csv 파일 업데이트
submission_path = '/content/drive/MyDrive/Colab Notebooks/Toy/21~23년/230911 MAIC ECG AI Challenge 2023/submission.csv'
submission_df = pd.read_csv(submission_path)

for idx, file_name in enumerate(test_files):
    submission_df.loc[submission_df['FILENAME'] == file_name, 'AGE'] = y_pred_rounded[idx]

submission_df.to_csv(submission_path, index=False)

35/35 [==============================] - 0s 4ms/step
